# Dialogflow API로 대화하기

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import urllib

In [2]:
class ChatBot():
    
    
    def __init__(self, key):
        self.key = key
        
        
    def run(self):
        while True:
            userInp = input('\n->')
            if userInp == 'quit':
                break
            resParams = self.getResponseParams(userInp)
            if resParams['aic'] == False:
                if resParams['ittName'] == 'query':
                    resParams['ans'] = self.answerQuery(resParams['params']['any'])
                if resParams['ittName'] == 'weather':
                    resParams['ans'] = self.answerWeather(resParams['params']['geo-city'])
            print(resParams['ans'])
            
            
    def getResponseParams(self, userInp):
        res = self.getResponse(userInp)
        
        return {'params': res['result']['parameters'],
                 'aic': res['result']['actionIncomplete'],
                 'ans': res['result']['fulfillment']['speech'],
                 'ittName': res['result']['metadata']['intentName']}
            
        
    def getResponse(self, userInp):
        data_send = {'query': userInp, 'sessionId': 'user01',
                     'lang': 'ko', 'timezone' : 'Asia/Seoul'}
        data_header = {'Authorization': 'Bearer %s'%self.key,
                       'Content-Type': 'application/json; charset=utf-8'}
        dialogflow_url = 'https://api.dialogflow.com/v1/query?v=20150910'
        res = requests.post(dialogflow_url, data=json.dumps(data_send), headers=data_header)
        
        if res.status_code == requests.codes.ok:
           return res.json()
    
    
    def answerQuery(self, word):
        url = 'https://search.naver.com/search.naver?where=kdic&query='
        url = url + urllib.parse.quote_plus(word)
        bs = BeautifulSoup(urllib.request.urlopen(url).read(), 'html.parser')
        outputs = bs.select('p.txt_box')
        
        return outputs[0].text
    
    
    def answerWeather(self, city):
        url = 'https://search.naver.com/search.naver?query='
        url = url + urllib.parse.quote_plus(city + '날씨')
        bs = BeautifulSoup(urllib.request.urlopen(url).read(), 'html.parser')
        temp = bs.select('span.todaytemp')[0].text
        desc = bs.select('p.cast_txt')[0].text
        
        return temp + ' / ' + desc

In [3]:
ChatBot('key').run()


->안녕
안녕!

->오늘 부산 날씨 알려줘
8 / 맑음, 어제보다 2˚ 높아요

->Python 알려줘
최근 몇 년 사이 프로그래밍을 비전공자들에게 알려주는 문화가 전세계적으로 확산되고 있다. 하버드나 예일대 같은 해외 대학 뿐만 아니라 카이스트, 국민대, 성균관대 등...

->quit
